# Controls & Data Quality Dashboard

**Purpose:** Instant-access reports for control effectiveness testing, data quality monitoring, and independent assurance tracking.

**Audience:** Chief Compliance Officer, Head of Internal Audit, Data Governance Officer, Control Testing Team, External Auditors

---

## The Audit Emergency Scenario

**Monday, 10:00 AM.** Internal Audit Director calls: *"External auditors arrive Wednesday. They need:**
1. **"How do we know our key compliance controls are working as designed?"**
2. **"Are there any critical data quality issues undermining regulatory reporting?"**
3. **"What independent assurance have we had over AML/sanctions/compliance recently?"**
4. **"Show us your control testing plan and results."**

**Traditional Response:** "I need a week to compile testing reports, data quality metrics, and audit logs from multiple systems..." (Auditors frustrated. Finding risk increases. Remediation costs escalate.)

**With This Notebook:** "I'll have the complete assurance pack ready by 5 PM today." (Runs 4 queries, exports testing results, pulls audit reports, delivers comprehensive evidence. Auditors impressed. Clean audit opinion maintained.)

---

## Daily Routine: Control & Data Quality Monitoring

### 8:00 AM - 8:30 AM: Daily Data Quality Dashboard Review
- Run Data Quality Dashboard
- Check completeness scores across all domains
- Flag any critical issues (completeness <90%)
- Review Control Effectiveness Dashboard
- Verify all key controls remain effective

### 9:00 AM - 12:00 PM: Control Testing & Validation
- **Monday:** Transaction monitoring control testing (sample 50 alerts)
- **Tuesday:** Customer screening control testing (sample 30 onboardings)
- **Wednesday:** Data quality validation (completeness/accuracy checks)
- **Thursday:** Regulatory reporting reconciliation (BCBS 239, Basel III)
- **Friday:** Access control review (user access logs, privileged access)

### Afternoon: Assurance Activity Management
- Coordinate audit/review schedules
- Provide evidence to auditors
- Track audit findings and management responses
- Monitor remediation action completion

---

## 1. Environment Setup

**Business Context:** Establish connection to control testing and data quality platform.

**Data Sources:**
- `CRMA_AGG_DT_CUSTOMER_CURRENT` - Customer data quality
- `ACCA_AGG_DT_ACCOUNTS` - Account data quality
- `PAYA_AGG_DT_TRANSACTION_ANOMALIES` - Transaction data completeness


In [ ]:
# Setup
from snowflake.snowpark.context import get_active_session
import streamlit as st
import pandas as pd
from datetime import datetime

session = get_active_session()
session.use_database('AAA_DEV_SYNTHETIC_BANK')
session.use_schema('CRM_AGG_001')
st.success('Connected to Data Quality Platform')

: 

## 2. Question 1: Data Quality Assessment

**Audit Question:** "Are there critical data quality issues that could undermine regulatory reporting?"

**Why This Matters:**
- **BCBS 239 Compliance:** Data quality is fundamental to risk reporting
- **Regulatory Risk:** Poor data quality undermines compliance programs
- **Operational Efficiency:** Clean data reduces false positives and manual work

**What We're Measuring:**
- Customer data completeness
- Missing critical fields
- Data validation failures
- Duplicate records


In [ ]:
# Query: Data Quality Metrics
query = '''
SELECT 
    COUNT(*) as total_customers,
    COUNT(CASE WHEN FIRST_NAME IS NULL THEN 1 END) as missing_first_name,
    COUNT(CASE WHEN FAMILY_NAME IS NULL THEN 1 END) as missing_family_name,
    COUNT(CASE WHEN EMAIL IS NULL THEN 1 END) as missing_email,
    COUNT(CASE WHEN DATE_OF_BIRTH IS NULL THEN 1 END) as missing_dob,

    ROUND((COUNT(*) - COUNT(CASE WHEN FIRST_NAME IS NULL OR FAMILY_NAME IS NULL 
                                    OR EMAIL IS NULL OR DATE_OF_BIRTH IS NULL 
THEN 1 END)) 
          * 100.0 / COUNT(*), 2) as completeness_percentage
FROM CRM_AGG_001.CRMA_AGG_DT_CUSTOMER_CURRENT
'''

df = session.sql(query).to_pandas()
st.subheader('Customer Data Quality Metrics')
st.dataframe(df, use_container_width=True)

## 3. Data Quality Dashboard

**Business Context:** BCBS 239 data quality validation for risk reporting.


In [ ]:
# Metrics
total = df['TOTAL_CUSTOMERS'].iloc[0]
completeness = df['COMPLETENESS_PERCENTAGE'].iloc[0]
missing_name = df['MISSING_FIRST_NAME'].iloc[0] + df['MISSING_FAMILY_NAME'].iloc[0]
missing_email = df['MISSING_EMAIL'].iloc[0]
missing_dob = df['MISSING_DOB'].iloc[0]

st.subheader('Data Quality Assessment')
col1, col2 = st.columns(2)

with col1:
    st.metric('Total Customers', f'{total:,}')
    st.metric('Data Completeness', f'{completeness:.1f}%')
    if completeness >= 99:
        st.success('Excellent data quality')
    elif completeness >= 95:
        st.warning('Good data quality - minor issues')
    else:
        st.error('Data quality needs improvement')

with col2:
    st.markdown('**Missing Critical Fields:**')
    st.metric('Missing Names', f'{missing_name:,}')
    st.metric('Missing Email', f'{missing_email:,}')
    st.metric('Missing DOB', f'{missing_dob:,}')

st.info('''
**BCBS 239 Data Quality Principles:**
- Accuracy: Data should correctly reflect business activities
- Completeness: All critical fields must be populated
- Timeliness: Data should be available when needed
- Integrity: Data should be consistent across systems
''')

## 4. Export Quality Report

**Purpose:** Generate data quality evidence for audit and regulatory validation.


In [ ]:
# Export
ts = datetime.now().strftime('%Y%m%d_%H%M%S')
filename = f'Data_Quality_Assessment_{ts}.csv'

export_df = df.copy()
export_df['REPORT_DATE'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
export_df['PREPARED_BY'] = 'Data Governance Officer / CCO'
export_df['FRAMEWORK'] = 'BCBS 239 Data Quality Principles'

csv_data = export_df.to_csv(index=False)

st.download_button(
    label='Download Quality Report (CSV)',
    data=csv_data,
    file_name=filename,
    mime='text/csv'
)

st.success(f'Data quality report ready: {filename}')

## Key Takeaways for CCO

**Always have these 3 reports ready:**
1. Control Effectiveness Testing Dashboard
2. Data Quality Dashboard
3. Independent Assurance Summary

**Response time targets:**
- Auditor request: <2 hours for control testing evidence
- Regulator inquiry: <4 hours for DQ metrics
- Board reporting: Quarterly assurance summary

**Key metrics to monitor:**
- **Control effectiveness:** >90% of controls rated effective
- **Data completeness:** >95% for all compliance-critical fields
- **Data accuracy:** >98% for all regulatory reporting fields
- **Action completion:** >80% of audit findings remediated on time

**Escalation Protocols:**

### Level 1: Daily Escalation
**Triggers:** Critical data quality issue (completeness <90%), Key control rated ineffective, Control testing failure (>3 consecutive failures)
**Action:** Email CCO + Control Owner with issue details + remediation plan

### Level 2: Weekly Escalation (Friday EOB)
**Triggers:** Multiple controls degraded, Data quality trending downward for 3+ weeks, Audit finding overdue by >30 days
**Action:** Executive committee meeting with remediation roadmap

### Level 3: Board Escalation
**Triggers:** Material audit finding (critical severity), Regulatory inspection with adverse rating, Control environment rated "weak" or "limited assurance"
**Action:** Board presentation with root cause analysis + comprehensive remediation plan + external advisor engagement

**Proactive Monitoring:**
- Run DQ dashboard daily (8:00 AM)
- Review control effectiveness weekly (Friday)
- Update assurance tracker monthly (end of month)
- Board assurance reporting quarterly

---

**Last Updated:** December 2025  
**Owner:** Chief Compliance Officer  
**Reviewers:** Head of Internal Audit, Chief Data Officer, Control Testing Manager